In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import array
from keras.models import load_model

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os

In [2]:
#read all the data:
items = pd.read_csv("item_features.csv")
purchase =  pd.read_csv("train_purchases.csv")
sessions =  pd.read_csv("train_sessions.csv")

In [3]:
import time
purchase['date'] = pd.to_datetime(purchase['date'])
sessions['date'] = pd.to_datetime(sessions['date'])
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [4]:
max_day = sessions['date'].max() #max date 31/05/2021 
testing_date = max_day -datetime.timedelta(31)
cond_test = sessions['date'] >= testing_date
cond_train = sessions['date'] < testing_date
cond_test_purchase = purchase.date >= testing_date
cond_train_purchase = purchase.date < testing_date
sessions_train = sessions[cond_train]
sessions_test = sessions[cond_test]
purchase_train = purchase[cond_train_purchase]
purchase_test = purchase[cond_test_purchase]

In [5]:
def sessions_list(sessions,purchase):
    antique_session = sessions.iat[0,0]
    list_sessions = []
    actual_session = []
    purchase_session = 0
    for i in tqdm(range(len(sessions))):
        new_session = sessions.iat[i,0]
        item = sessions.iat[i,1]
        if(new_session != antique_session):
            actual_session.append(purchase.iat[purchase_session,1])
            purchase_session += 1
            list_sessions.append(actual_session)
            actual_session = []
            antique_session = new_session
        actual_session.append(item)
    list_sessions.append(actual_session)
    return list_sessions

In [6]:
session_item = sessions_list(sessions,purchase)

100%|██████████| 4743820/4743820 [03:45<00:00, 21063.26it/s]


In [7]:
def training_sequences(session_item2):
    train_sequence = np.zeros((len(session_item),6))
    trobat = False
    for i,x in tqdm(enumerate(session_item2)):
        length = len(x)
        llista = x.copy()
        if(length-1 < 5):
            for j in range(5-length+1):
                llista.insert(0,0)
        if(length-1 > 5):
            for j in range(length-6):
                llista.pop(0)
        llista.pop(-1)                                    
        train_sequence[i][0] = llista[0]
        train_sequence[i][1] = llista[1]
        train_sequence[i][2] = llista[2]
        train_sequence[i][3] = llista[3]
        train_sequence[i][4] = llista[4]
        train_sequence[i][5] = x[len(x)-1]
    return train_sequence

sequence_training = training_sequences(session_item)

1000000it [00:03, 305268.88it/s]


In [8]:
X = np.array(sequence_training)[:,:5] #sequence to predict from
y = np.array(sequence_training)[:,5:] #next item in the sequence
X = X.astype(int)
y = y.astype(int)
training = np.array(sequence_training)

# LSTM:

In [9]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [10]:
number_items = len(items.item_id.unique())
number_items += 1
number_items

23692

In [11]:
item_index = {}
index_item = {}
item_index[0] = 0
index_item[0] = 0
for i,x in enumerate(items.item_id.unique()):
    index_item[i+1] = x
    item_index[x] = i+1

In [12]:
def training_dataset(X2,y2):
    #first X.
    X_training = np.zeros((X2.shape[0],X2.shape[1]))
    y_training = np.zeros(y2.shape[0])
    
    for i in range(X2.shape[0]):
        for j in range(X2.shape[1]):
            X_training[i,j] = item_index[X2[i,j]]
    
    for i in range(y2.shape[0]):
        y_training[i] = item_index[y2[i][0]]
            
    return X_training, y_training
X_training, y_training = training_dataset(X,y)  
X_training = X_training.astype(int)
y_training = y_training.astype(int)

In [33]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    
    return model

In [34]:
model = create_model(6, number_items)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 10)             236920    
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 23692)             2392892   
                                                                 
Total params: 2,674,212
Trainable params: 2,674,212
Non-trainable params: 0
_________________________________________________________________


In [35]:
#let's define a sparse categorical crossentropy to start!
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

In [58]:
#let's define a sparse categorical crossentropy to start!
def loss2(labels, logits):
  print(labels)
  labels = int(labels)
  index = item_index[labels]
  return 1
  logits2 = np.array(logits)
  return 1
  res = list(sorted(enumerate(test_list)))[-100:]
  for i in range(100):
    if(index == res[i][0]): return 1
  return 1

In [60]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam_optimizer,
    loss=loss
)

In [59]:
model.fit(X_training,y_training,epochs=20, verbose=1)

Epoch 1/20


TypeError: ignored

In [44]:
from operator import itemgetter

test_list = [1,2,3,4,5,6]
K = 2
res = list(sorted(enumerate(test_list)))[-K:]
res

[(4, 5), (5, 6)]

In [ ]:
vec = [1,2,3,4,5]
sorted(np.vstack((range(5),vec)))

ValueError: ignored

In [32]:
model.predict(X_training[0].reshape(1,5))

array([[1.15590915e-07, 1.15554101e-07, 5.46083829e-05, ...,
        5.73711841e-05, 1.14773769e-07, 2.73790715e-06]], dtype=float32)

In [25]:
X_training[0]

array([   0,    0,    0, 8123, 8123])